### Optimize Drag with Neural Networks and NSGA2 ###

In [1]:
# import the fun
import sys
import numpy as np
from tqdm import tqdm
import math
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import Guided_Cond_DDPM_Tools as GC_DDPM

import Optimization_Tools as OT

from sklearn.decomposition import PCA

from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.core.problem import Problem
from pymoo.optimize import minimize



sys.path.append('/home/ada/Documents/HullParameterization')
from HullParameterization import Hull_Parameterization as HP

np.set_printoptions(suppress=True) # don't use scientific notation

In [2]:
#Upload important info
Reg_PATHS = ['./TrainedModels/Regressor_CT.pth',
        './TrainedModels/Regressor_LOA_wBulb.pth',
        './TrainedModels/Regressor_WL.pth']

DesVec_82k = np.load('DesVec_82k.npy', allow_pickle=True)
print(DesVec_82k.shape)

X_LIMITS = np.zeros((len(DesVec_82k[0]), 2))

X_LIMITS[1:] = np.load('/home/ada/Documents/HullParameterization/HullDiffusion/Restructured_Dataset/X_LIMITS.npy')
print(X_LIMITS.shape)

(82168, 45)
(45, 2)


In [3]:
# Regression Model Setup

nodes = 512
Drag_Reg_Dict = {
        'xdim' : 44,              # Dimension of parametric design vector
        'ydim': 1,                              # trains regression model for each objective
        'tdim': nodes,                            # dimension of latent variable
        'net': [nodes,nodes,nodes,nodes],                       # network architecture        
        'Training_Epochs': 50000,  #30000             # number of training epochs
        'batch_size': 1024,                       # batch size
        'Model_Label': 'Regressor_CT',         # labels for regressors       
        'lr' : 0.001,                          # learning rate
        'weight_decay': 0.0,                   # weight decay
        'device_name': 'cuda:0'} 

nodes = 256
LOA_wBulb_Reg_Dict = {
        'xdim' : 44,              # Dimension of parametric design vector
        'ydim': 1,                              # trains regression model for each objective
        'tdim': nodes,                            # dimension of latent variable
        'net': [nodes,nodes,nodes],                       # network architecture        
        'Training_Epochs': 150000,               # number of training epochs
        'batch_size': 1024,                       # batch size
        'Model_Label': 'Regressor_LOA_wBulb',         # labels for regressors
                    
        'lr' : 0.001,                          # learning rate
        'weight_decay': 0.0,                   # weight decay
        'device_name': 'cuda:0'}   

WL_Reg_Dict = {
        "xdim": 45,
        "ydim": 1, 
        "tdim": 512, 
        "net": [512, 512, 512], 
        "Training_Epochs": 30000, 
        "batch_size": 1024, 
        "Model_Label": 
        "Regressor_WL", 
        "lr": 0.001, 
        "weight_decay": 0.0, 
        "device_name": "cuda:0"}

In [4]:
'''
Xopt = res.pop.get('X')
Fopt = res.pop.get('F')

#Xopt_seeded = res2.pop.get('X')
#Fopt_seeded = res2.pop.get('F')



FminHist =  [e.opt.get("F")[0] for e in res.history]
#FminHist_seeded =  [e.opt.get("F")[0] for e in res2.history]

plt.ylim([0,2e7])
plt.plot(np.arange(len(FminHist)), FminHist)
#plt.plot(np.arange(len(FminHist_seeded)), FminHist_seeded)

plt.show()


#print(FminHist[-1])
'''




'\nXopt = res.pop.get(\'X\')\nFopt = res.pop.get(\'F\')\n\n#Xopt_seeded = res2.pop.get(\'X\')\n#Fopt_seeded = res2.pop.get(\'F\')\n\n\n\nFminHist =  [e.opt.get("F")[0] for e in res.history]\n#FminHist_seeded =  [e.opt.get("F")[0] for e in res2.history]\n\nplt.ylim([0,2e7])\nplt.plot(np.arange(len(FminHist)), FminHist)\n#plt.plot(np.arange(len(FminHist_seeded)), FminHist_seeded)\n\nplt.show()\n\n\n#print(FminHist[-1])\n'

In [5]:
'''
from sklearn.decomposition import PCA
pca = PCA(n_components = 2)

pca.fit(DesVec_82k[:,1:])
X1 = x_samples[valid_idx]

A = np.random.randint(0,len(DesVec_82k),100)
B = np.random.randint(0,len(X1),100)

components_dataset = pca.transform(DesVec_82k[A,1:])

comp = pca.transform(X1[B,1:])



plt.rcParams['font.family'] = 'serif'

plt.rcParams['font.size'] = '8' 


fig, axs = plt.subplots(1, 1) #, sharex='all',sharey='all')
fig.figsize = (4.5,3)
fig.dpi = 300

title = 'PCA Distribution of Generated Parameters'

  


#plt.xlim([-40,40])
#plt.ylim([-40,40])
plt.title = title
plt.xticks( fontsize=8)
plt.yticks( fontsize=8)



axs.spines["right"].set_visible(False)
axs.spines["top"].set_visible(False)
axs.set_title(title, fontsize = 8)

axs.plot(components_dataset[:,0], components_dataset[:,1], 'o', color = (1,0.5,0.5), alpha=0.5, markersize=5, label = 'Dataset')

axs.plot(comp[:,0], comp[:,1], 'o', color = (0.2,0.2,1), markeredgecolor='black', alpha= 1, markersize=5, label = 'Interpolated Samples')

    

axs.set_xlabel('X0', fontsize = 8)

axs.set_ylabel('X1', fontsize = 8)

fig.tight_layout(pad=1.25)
axs.legend(fontsize = 8, loc = (0.75,-.15),frameon=False)

#figpath = path + 'Graphs_and_Images/Interp_Study_PCA.png'
#plt.savefig(figpath)

plt.show()
'''

'\nfrom sklearn.decomposition import PCA\npca = PCA(n_components = 2)\n\npca.fit(DesVec_82k[:,1:])\nX1 = x_samples[valid_idx]\n\nA = np.random.randint(0,len(DesVec_82k),100)\nB = np.random.randint(0,len(X1),100)\n\ncomponents_dataset = pca.transform(DesVec_82k[A,1:])\n\ncomp = pca.transform(X1[B,1:])\n\n\n\nplt.rcParams[\'font.family\'] = \'serif\'\n\nplt.rcParams[\'font.size\'] = \'8\' \n\n\nfig, axs = plt.subplots(1, 1) #, sharex=\'all\',sharey=\'all\')\nfig.figsize = (4.5,3)\nfig.dpi = 300\n\ntitle = \'PCA Distribution of Generated Parameters\'\n\n  \n\n\n#plt.xlim([-40,40])\n#plt.ylim([-40,40])\nplt.title = title\nplt.xticks( fontsize=8)\nplt.yticks( fontsize=8)\n\n\n\naxs.spines["right"].set_visible(False)\naxs.spines["top"].set_visible(False)\naxs.set_title(title, fontsize = 8)\n\naxs.plot(components_dataset[:,0], components_dataset[:,1], \'o\', color = (1,0.5,0.5), alpha=0.5, markersize=5, label = \'Dataset\')\n\naxs.plot(comp[:,0], comp[:,1], \'o\', color = (0.2,0.2,1), markere

In [6]:
Ships = np.array([[333, 42.624, 11.28, 29.064, 97561, 16], #Nimitz Class Carrier [LOA(m), BOA(m), T(m), Dd(m), Vol(m^3), U(m/s)] 
                  [3.8, .787, .15, .438, .166, 1.5], #Kayak [LOA(m), BOA(m), T(m), Dd(m), Vol(m^3), U(m/s)]
                  [366, 50, 15.2, 40, 182114, 10.3], #Neo-Panamax Container Ship [LOA(m), BOA(m), T(m), Dd(m), Vol(m^3), U(m/s)]
                  [127, 16,6.9,11, 4488, 14.4], #NSC [LOA(m), BOA(m), T(m), Dd(m), Vol(m^3), U(m/s)]
                  [72,20,3.2,4.8, 3917, 6.17] #ROPAX ferry [LOA(m), BOA(m), T(m), Dd(m), Vol(m^3), U(m/s)]
                  ])

Labels = ['Nimitz', 'Kayak', 'Neo-Panamax Container Ship', 'NSC', 'ROPAX ferry']

In [7]:
# Set up the Problem:
popSize = 100
Gen = 200

#popSize = 250 # trying to get the ROPAX to optimize

#for i in range(0,len(Ships)):
for i in [4]: #,4] #range(4,5):


    print('Starting Study {}'.format(i))
    
    Study_Label = 'Study_' + str(i)
    
    LOA = Ships[i,0]
    BOA = Ships[i,1]
    Draft = Ships[i,2]
    Depth = Ships[i,3]
    Vol = Ships[i,4]

    U = Ships[i,5] #12.86 #m/s  = 25 knots

    


    Geom_Cond = [LOA,
                BOA,
                Draft,
                Depth,
                Vol,
                U, 
                Gen]

    X_LIMITS = np.zeros((len(DesVec_82k[0]), 2))

    X_LIMITS[1:] = np.load('/home/ada/Documents/HullParameterization/HullDiffusion/Restructured_Dataset/X_LIMITS.npy')
    print(X_LIMITS.shape)

    X_LIMITS[0,0:2] = [LOA*0.8, LOA*1.000000001]


    problem = OT.Minimize_Drag(Geom_Cond,Reg_PATHS,X_LIMITS,DesVec_82k[:,1:], Drag_Reg_Dict,LOA_wBulb_Reg_Dict,WL_Reg_Dict, 'cuda:0')

    #Pop = OT.gen_Pop(popSize, X_LIMITS[:,0], X_LIMITS[:,1])

    A = np.random.randint(0,len(DesVec_82k),popSize)

    Pop = np.zeros((popSize,len(DesVec_82k[0])))

    


    Pop[:,1:] = DesVec_82k[A,1:] #Initialize population with random designs from dataset
    Pop[:,0] = np.random.random(popSize)*(X_LIMITS[0,1]-X_LIMITS[0,0])+X_LIMITS[0,0] #Initialize population with random designs from dataset

    X_DDPM = np.load('./Optimization_Study_Samples/' + Study_Label + '/DDPM_DesVec.npy')
    A = np.random.randint(0,len(X_DDPM),32)

    Pop[0:32] = X_DDPM[A] #Initialize population with random designs from dataset


    

    print(Pop.shape)

    constraints = []
    sum_violation = []
    cons = []
    valid_idx = []

    for i in tqdm(range(0,len(Pop))):
        hull = HP(Pop[i])
        constraints.append(hull.input_Constraints())
        cons.append(constraints[i] > 0)
        
        if sum(cons[i]) == 0:
            valid_idx.append(i)
            #hull.Calc_VolumeProperties(NUM_WL = 101, PointsPerWL = 1000)
        sum_violation.append(sum(cons[i]))

    print(len(valid_idx))

    print('Starting Optimization')
    algorithm = NSGA2(
            pop_size=popSize,
            sampling = Pop,
            eliminate_duplicates=True)

    res = minimize(problem,
                algorithm,
                ('n_gen', Gen),
                seed=1,
                verbose=True,
                save_history=True)
    
    #print('Starting Seeded Optimization')
    
    Xopt = res.pop.get('X')
    Fopt = res.pop.get('F')

    #Xopt_seeded = res2.pop.get('X')
    #Fopt_seeded = res2.pop.get('F')

    DesVec_Opt = Xopt.copy()

    DesVec_Opt = OT.clean_designVector(DesVec_Opt)

    x_norm = problem.D.data_norm.transform_Data(DesVec_Opt[:,1:])

    drag_cond = np.repeat(problem.dim, len(Xopt), axis=0)

    Rt,Ct,Fn = problem.D.Predict_Drag(x_norm,drag_cond, DesVec_Opt[:,0:1])

    print(np.mean(Rt))

    np.save('./Optimization_Study_Samples/' + Study_Label + '/OptimalDrag_Rt_NNReg.npy',Rt)

    np.save('./Optimization_Study_Samples/' + Study_Label + '/OptimalDrag_DesVec.npy',DesVec_Opt)


    # Generate some STLs


    print(len(valid_idx))
    sample_vol = []
    sample_BOA = []
    sample_Dd = []
    idx_to_remove = []

    for i in tqdm(range(0,len(Xopt))):
        hull = HP(Xopt[i]) 
        #print(i)
        try:
            Z = hull.Calc_VolumeProperties(NUM_WL = 101, PointsPerWL = 1000)    
            sample_vol.append(HP.interp(hull.Volumes, Z, Draft))
            Beam = max(hull.Calc_Max_Beam_midship(), hull.Calc_Max_Beam_PC())
            sample_BOA.append(Beam)
            sample_Dd.append(hull.Dd)
        except:
            print('error at hull {}'.format(i))
            idx_to_remove.append(i)

            continue

    sample_vol = np.array(sample_vol)
    sample_BOA = np.array(sample_BOA)
    sample_Dd = np.array(sample_Dd)

    VolMEAP = np.mean(np.abs(sample_vol - Vol)/Vol)*100
    print('Volume MEAP: {}%'.format(VolMEAP))

    BOAMEAP = np.mean(np.abs(sample_BOA - BOA)/BOA)*100
    print('Beam MEAP: {}%'.format(BOAMEAP))

    DdMEAP = np.mean(np.abs(sample_Dd - Depth)/Depth)*100
    print('Depth MEAP: {}%'.format(DdMEAP))


    for i in tqdm(range(0,32)):
        Hull = HP(Xopt[i])
        #Hull_seeded = HP(Xopt_seeded[i])

        #Check Constriants:
        constraints = Hull.input_Constraints()
        cons = constraints > 0
        #print(sum(cons)) # should be zero

        #make the .stl file of the hull:
        try:

            strpath =  './Optimization_Study_Samples/' + Study_Label + '/stl_opt/opt_Hull_'  + str(i)
            mesh = Hull.gen_stl(NUM_WL=100, PointsPerWL=800, bit_AddTransom = 1, bit_AddDeckLid = 1, namepath = strpath)
        except:
            print('error at hull {}'.format(i))
            continue


        #strpath_seeded =  './Optimization_Study_Samples/' + Study_Label + '/stl_opt/SEEDED_opt_Hull_'  + str(i)
        #mesh = Hull_seeded.gen_stl(NUM_WL=100, PointsPerWL=800, bit_AddTransom = 1, bit_AddDeckLid = 1, namepath = strpath)



Starting Study 4
(45, 2)
(100, 45)


100%|██████████| 100/100 [00:00<00:00, 11538.35it/s]

100
Starting Optimization


n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |       98 |      1 |  4.305770E+01 |  2.066332E+03 |             - |             -


/home/ada/anaconda3/lib/python3.11/site-packages/pymoo/operators/crossover/sbx.py:47: RuntimeWarning: invalid value encountered in power
  betaq[mask] = np.power((rand * alpha), (1.0 / (eta + 1.0)))[mask]


...

error at hull 26
error at hull 27


 94%|█████████▍| 30/32 [00:03<00:00,  8.01it/s]

error at hull 28
error at hull 29


100%|██████████| 32/32 [00:04<00:00,  7.74it/s]

error at hull 30
error at hull 31
